In [99]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,18 hours 23 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_paaske_rcki3a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.506 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [100]:
# Reading csv to h2o dataframe 
col_types_train = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'revenue': 'int',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int',
 'municipality_size_group': 'enum',
 'mean_revenue_for_municipality_size_group': 'int',
 'median_revenue_for_municipality_size_group': 'int',
 'st_dev_of_revenue_for_municipality_size_group': 'int'}

col_types_test = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int',
 'municipality_size_group': 'enum',
 'mean_revenue_for_municipality_size_group': 'int',
 'median_revenue_for_municipality_size_group': 'int',
 'st_dev_of_revenue_for_municipality_size_group': 'int'}

 
train_data = h2o.import_file('feature_data/training_set_dropped.csv', col_types = col_types_train)
test_data = h2o.import_file('feature_data/testing_set_dropped.csv', col_types = col_types_test)

test = pd.read_csv('feature_data/testing_set_dropped.csv')

# Save store ids, then remove this column
store_ids = test['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True )

train_data.types



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


{'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'revenue': 'real',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'real',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int',
 'closest_competitor': 'int',
 'num_stores_within_100m_and_same_lvl3': 'int',
 'num_stores_within_500m_and_same_lvl3': 'int',
 'num_stores_within_1km_and_same_lvl3': 'int',
 'num_stores_within_5km_and_same_lvl3': 'int',
 'num_stores_within_10km_and_same_lvl3': 'int',
 'num_stores_with

In [101]:
# Setting features as factor
""" train_data['store_id'] = train_data['store_id'].asfactor()
train_data['chain_name'] = train_data['chain_name'].asfactor()
train_data['mall_name'] = train_data['mall_name'].asfactor()
train_data['revenue'] = train_data['revenue'].asfactor()
train_data['lv1'] = train_data['lv1'].asfactor()
train_data['lv2'] = train_data['lv2'].asfactor()
train_data['lv3'] = train_data['lv3'].asfactor()
train_data['total_nbr_people'] = train_data['total_nbr_people'].asfactor()
train_data['nbr_people_per_store_in_grunnkrets'] = train_data['nbr_people_per_store_in_grunnkrets'].asfactor()
train_data['counts_gr_lv2'] = train_data['counts_gr_lv2'].asfactor()
train_data['counts_municipality_lv2'] = train_data['counts_municipality_lv2'].asfactor()
train_data['busstops_within_50m'] = train_data['busstops_within_50m'].asfactor()
train_data['busstops_within_100m'] = train_data['busstops_within_100m'].asfactor()
train_data['busstops_within_400m'] = train_data['busstops_within_400m'].asfactor()
train_data['busstops_within_800m'] = train_data['busstops_within_800m'].asfactor()
train_data['busstops_within_1500m'] = train_data['busstops_within_1500m'].asfactor()
train_data['num_stores_within_100m_and_same_lvl2'] = train_data['num_stores_within_100m_and_same_lvl2'].asfactor()
train_data['num_stores_within_500m_and_same_lvl2'] = train_data['num_stores_within_500m_and_same_lvl2'].asfactor()
train_data['num_stores_within_1km_and_same_lvl2'] = train_data['num_stores_within_1km_and_same_lvl2'].asfactor()
train_data['num_stores_within_5km_and_same_lvl2'] = train_data['num_stores_within_5km_and_same_lvl2'].asfactor()
train_data['num_stores_within_10km_and_same_lvl2'] = train_data['num_stores_within_10km_and_same_lvl2'].asfactor()
train_data['num_stores_within_20km_and_same_lvl2'] = train_data['num_stores_within_20km_and_same_lvl2'].asfactor() """

" train_data['store_id'] = train_data['store_id'].asfactor()\ntrain_data['chain_name'] = train_data['chain_name'].asfactor()\ntrain_data['mall_name'] = train_data['mall_name'].asfactor()\ntrain_data['revenue'] = train_data['revenue'].asfactor()\ntrain_data['lv1'] = train_data['lv1'].asfactor()\ntrain_data['lv2'] = train_data['lv2'].asfactor()\ntrain_data['lv3'] = train_data['lv3'].asfactor()\ntrain_data['total_nbr_people'] = train_data['total_nbr_people'].asfactor()\ntrain_data['nbr_people_per_store_in_grunnkrets'] = train_data['nbr_people_per_store_in_grunnkrets'].asfactor()\ntrain_data['counts_gr_lv2'] = train_data['counts_gr_lv2'].asfactor()\ntrain_data['counts_municipality_lv2'] = train_data['counts_municipality_lv2'].asfactor()\ntrain_data['busstops_within_50m'] = train_data['busstops_within_50m'].asfactor()\ntrain_data['busstops_within_100m'] = train_data['busstops_within_100m'].asfactor()\ntrain_data['busstops_within_400m'] = train_data['busstops_within_400m'].asfactor()\ntrain_

In [102]:
#test_data['revenue'] = test_data['revenue'].asfactor()
""" test_data['store_id'] = test_data['store_id'].asfactor()
test_data['chain_name'] = test_data['chain_name'].asfactor()
test_data['mall_name'] = test_data['mall_name'].asfactor()
test_data['lv1'] = test_data['lv1'].asfactor()
test_data['lv2'] = test_data['lv2'].asfactor()
test_data['lv3'] = test_data['lv3'].asfactor()
test_data['total_nbr_people'] = test_data['total_nbr_people'].asfactor()
test_data['nbr_people_per_store_in_grunnkrets'] = test_data['nbr_people_per_store_in_grunnkrets'].asfactor()
test_data['counts_gr_lv2'] = test_data['counts_gr_lv2'].asfactor()
test_data['counts_municipality_lv2'] = test_data['counts_municipality_lv2'].asfactor()
test_data['busstops_within_50m'] = test_data['busstops_within_50m'].asfactor()
test_data['busstops_within_100m'] = test_data['busstops_within_100m'].asfactor()
test_data['busstops_within_400m'] = test_data['busstops_within_400m'].asfactor()
test_data['busstops_within_800m'] = test_data['busstops_within_800m'].asfactor()
test_data['busstops_within_1500m'] = test_data['busstops_within_1500m'].asfactor()
test_data['num_stores_within_100m_and_same_lvl2'] = test_data['num_stores_within_100m_and_same_lvl2'].asfactor()
test_data['num_stores_within_500m_and_same_lvl2'] = test_data['num_stores_within_500m_and_same_lvl2'].asfactor()
test_data['num_stores_within_1km_and_same_lvl2'] = test_data['num_stores_within_1km_and_same_lvl2'].asfactor()
test_data['num_stores_within_5km_and_same_lvl2'] = test_data['num_stores_within_5km_and_same_lvl2'].asfactor()
test_data['num_stores_within_10km_and_same_lvl2'] = test_data['num_stores_within_10km_and_same_lvl2'].asfactor()
test_data['num_stores_within_20km_and_same_lvl2'] = test_data['num_stores_within_20km_and_same_lvl2'].asfactor() """

" test_data['store_id'] = test_data['store_id'].asfactor()\ntest_data['chain_name'] = test_data['chain_name'].asfactor()\ntest_data['mall_name'] = test_data['mall_name'].asfactor()\ntest_data['lv1'] = test_data['lv1'].asfactor()\ntest_data['lv2'] = test_data['lv2'].asfactor()\ntest_data['lv3'] = test_data['lv3'].asfactor()\ntest_data['total_nbr_people'] = test_data['total_nbr_people'].asfactor()\ntest_data['nbr_people_per_store_in_grunnkrets'] = test_data['nbr_people_per_store_in_grunnkrets'].asfactor()\ntest_data['counts_gr_lv2'] = test_data['counts_gr_lv2'].asfactor()\ntest_data['counts_municipality_lv2'] = test_data['counts_municipality_lv2'].asfactor()\ntest_data['busstops_within_50m'] = test_data['busstops_within_50m'].asfactor()\ntest_data['busstops_within_100m'] = test_data['busstops_within_100m'].asfactor()\ntest_data['busstops_within_400m'] = test_data['busstops_within_400m'].asfactor()\ntest_data['busstops_within_800m'] = test_data['busstops_within_800m'].asfactor()\ntest_dat

In [103]:
# Removing labels from train set
y = "revenue"
x = train_data.columns
x.remove(y)

In [104]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x = x, y = y, training_frame = train_data)

AutoML progress: |
12:09:57.75: _train param, Dropping bad and constant columns: [closest_competitor]

███
12:10:10.145: _train param, Dropping bad and constant columns: [closest_competitor]

█
12:10:23.103: _train param, Dropping bad and constant columns: [closest_competitor]
12:10:27.190: _train param, Dropping bad and constant columns: [closest_competitor]

█
12:10:32.535: _train param, Dropping bad and constant columns: [closest_competitor]

███
12:11:32.918: _train param, Dropping bad and constant columns: [closest_competitor]
12:11:36.1: _train param, Dropping bad and constant columns: [closest_competitor]

█
12:11:39.992: _train param, Dropping bad and constant columns: [closest_competitor]

█
12:11:45.609: _train param, Dropping bad and constant columns: [closest_competitor]

██
12:11:53.58: _train param, Dropping bad and constant columns: [closest_competitor]

███
12:12:58.722: _train param, Dropping unused columns: [closest_competitor]
12:12:59.231: _train param, Dropping unu

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5008711,0.0109797,0.5047334,0.5082741,0.5114883,0.4954229,0.4844369
mean_residual_deviance,0.4436510,0.0125800,0.4508856,0.4555194,0.4515766,0.4325586,0.4277146
mse,0.4436510,0.0125800,0.4508856,0.4555194,0.4515766,0.4325586,0.4277146
null_deviance,2590.0974,93.278114,2629.1086,2624.795,2686.5435,2569.4382,2440.6023
r2,0.5668026,0.0091834,0.5570296,0.5586107,0.5676047,0.5717233,0.5790446
residual_deviance,1122.2753,59.2499,1163.7357,1158.386,1161.4551,1100.4291,1027.3704
rmse,0.6660175,0.0094686,0.6714801,0.6749218,0.6719945,0.6576919,0.6539989
rmsle,0.2829434,0.0043429,0.2842179,0.2887503,0.2841769,0.2773703,0.2802016


In [105]:
lb = aml.leaderboard

In [106]:
# Showing best perforimg models
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_4_20221109_120957,0.666238,0.443873,0.501055,0.283,0.443873
StackedEnsemble_BestOfFamily_1_AutoML_4_20221109_120957,0.666982,0.444865,0.50236,0.283456,0.444865
GBM_1_AutoML_4_20221109_120957,0.672445,0.452182,0.506048,0.284425,0.452182
GBM_2_AutoML_4_20221109_120957,0.67291,0.452808,0.505471,0.284487,0.452808
GBM_3_AutoML_4_20221109_120957,0.673476,0.453569,0.505871,0.284555,0.453569
DRF_1_AutoML_4_20221109_120957,0.678003,0.459688,0.508847,0.286617,0.459688
GBM_4_AutoML_4_20221109_120957,0.678658,0.460577,0.509759,0.286496,0.460577
XGBoost_3_AutoML_4_20221109_120957,0.710814,0.505257,0.54644,0.300959,0.505257
GLM_1_AutoML_4_20221109_120957,0.72284,0.522498,0.559994,0.304878,0.522498
XRT_1_AutoML_4_20221109_120957,0.725354,0.526138,0.567141,0.30309,0.526138


In [107]:
preds = aml.predict(test_data)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/paaske/.local/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'store_id' has levels not trained on: ["810130822-971657243-219275", "810182482-971658150-11075", "811480282-971683597-179256", "811604372-988862983-86893", "811618772-912015351-623892", "811625612-911659778-433802", "811661112-911677962-612425", "811665762-971687568-21394", "811672602-974079658-32805", "811673862-974995875-40151", ...8557 not listed..., "999649262-990192235-115914", "999649440-972257710-39400", "999653987-911550016-614706", "999655866-811552712-614673", "999657117-971794577-1512", "999662072-911698862-612468", "999665624-992886544-338735", "999666124--87921-87921", "999666728-911570688-517908", "999668615-980238296-585056"]
  warnings.warn(w)
/home/paaske/.local/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'chain_name' has levels not trained on: ["24 7 TRENINGSSENTER", "ACE SHOPS", "AVANCIA SPORT", "BADEMILJØ", "BADERINGE

In [108]:
#preds = h2o.as_list(preds)
print(preds)

  predict
  1.83032
  1.68012
  2.98683
  2.14364
  2.05322
  2.22204
  2.07244
  1.88467
  2.11908
  1.74444
[8578 rows x 1 column]



In [109]:
# Convert result to desired format

import pandas as pd
import numpy as np


pandas_preds = preds.as_data_frame()
pandas_preds.rename(columns={'predict' :'predicted'}, inplace=True)
pandas_preds['predicted'] = np.expm1(pandas_preds['predicted'])

def save_submission(pred):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('h2o.csv', index=False)
    
save_submission(pandas_preds)

